In [28]:
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
from pypdf import PdfReader
import gradio as gr
from pydantic import BaseModel
import os

In [29]:
pdfReader = PdfReader("Resources/Profile.pdf")
prof_summary = ""
for page in pdfReader.pages:
    text = page.extract_text()
    if text:
        prof_summary += text + "\n"
    display(Markdown(text))
print(prof_summary)

Parag Agrawal - Professional Profile
Summary
Parag Agrawal is an Indian-American software engineer and entrepreneur, best known for his tenure
as the CEO of Twitter from November 2021 to October 2022. He was appointed CEO following Jack
Dorsey's resignation but was dismissed after Elon Musk's acquisition of Twitter.
Education and Early Career
Agrawal holds a B.Tech in Computer Science and Engineering from the Indian Institute of
Technology (IIT) Bombay. He then pursued a Ph.D. in Computer Science at Stanford University,
where his research focused on uncertainty in data management and integration. Prior to his role at
Twitter, he held research internships at Microsoft Research and Yahoo! Research.
Career at Twitter
Agrawal joined Twitter in 2011 as a software engineer and quickly rose through the ranks. In 2017,
he was appointed Chief Technology Officer (CTO), overseeing Twitter's technical strategy and
leading initiatives like Project Bluesky, aimed at developing a decentralized social media protocol.
He succeeded Jack Dorsey as CEO in 2021, focusing on areas such as artificial intelligence,
machine learning, and decentralization.
Post-Twitter Venture: Parallel Web Systems
After leaving Twitter, Agrawal co-founded Parallel Web Systems Inc., an AI startup that exited
stealth mode in October 2024. The company has raised approximately $30 million in funding from
investors like Khosla Ventures, Index Ventures, and First Round Capital. Parallel Web Systems
aims to develop an API that enables AI systems to interact with web data, facilitating advanced AI
operations on behalf of users. The startup is based in Palo Alto, California, and employs a team of
around 10 individuals.
Personal Life
Agrawal is married to Vineeta Agarwala, a general partner at the venture capital firm Andreessen

Parag Agrawal - Professional Profile
Horowitz, where she focuses on investments in biotech and health tech. The couple resides in San
Francisco and has two children.

Parag Agrawal - Professional Profile
Summary
Parag Agrawal is an Indian-American software engineer and entrepreneur, best known for his tenure
as the CEO of Twitter from November 2021 to October 2022. He was appointed CEO following Jack
Dorsey's resignation but was dismissed after Elon Musk's acquisition of Twitter.
Education and Early Career
Agrawal holds a B.Tech in Computer Science and Engineering from the Indian Institute of
Technology (IIT) Bombay. He then pursued a Ph.D. in Computer Science at Stanford University,
where his research focused on uncertainty in data management and integration. Prior to his role at
Twitter, he held research internships at Microsoft Research and Yahoo! Research.
Career at Twitter
Agrawal joined Twitter in 2011 as a software engineer and quickly rose through the ranks. In 2017,
he was appointed Chief Technology Officer (CTO), overseeing Twitter's technical strategy and
leading initiatives like Project Bluesky, aimed at developing a decentralized social

In [30]:
with open("Resources/Summary.txt", "r", encoding="utf-8") as file:
    summary = file.read()

In [31]:
import os

load_dotenv(override=True)  

True

In [32]:

openai_api_key = os.getenv('API_TOKEN')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:14]}")
else:
    print("OpenAI API Key not set - please head to the troubleshooting guide in the setup folder")

OpenAI API Key exists and begins sk-or-v1-e268c


In [33]:
# System Prompt
name = "Parag Agrawal"

evaluator_system_prompt = (
    f"You are an evaluator assessing AI Agent responses.\n\n"
    f"The Agent represents {name} on their website. Evaluate if the response is acceptable based on:\n"
    f"1. **Accuracy** - Does it align with {name}'s actual background?\n"
    f"2. **Professionalism** - Is the tone appropriate?\n"
    f"3. **Completeness** - Does it answer the question?\n\n"
    f"## {name}'s Background:\n{summary}\n\n{prof_summary}\n\n"
    f"**Be reasonable**: Minor imperfections are OK. Only mark as unacceptable if the response:\n"
    f"- Contains factually incorrect information about {name}\n"
    f"- Is unprofessional or inappropriate\n"
    f"- Completely fails to address the question\n"
    f"- Is too vague when specific information is available\n\n"
    f"Most well-formed, relevant responses should be marked as **acceptable**.\n"
    f"Return: is_acceptable (true/false) and brief feedback."
)

system_prompt = (
    f"You are acting as {name}, representing {name} on their website. "
    f"Your role is to answer questions specifically about {name}'s career, background, skills, and experience. "
    f"You must faithfully and accurately portray {name} in all interactions. "
    f"You have access to a detailed summary of {name}'s background and their LinkedIn profile, which you should use to inform your answers. "
    f"Maintain a professional, engaging, and approachable tone, as if you are speaking to a potential client or future employer visiting the site. "
    f"If you are unsure of an answer, it is better to honestly acknowledge that than to guess."

    f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{prof_summary}\n\n"
    f"Using this context, please converse naturally and consistently, always staying in character as {name}."
)

In [34]:
evaluator_system_prompt

"You are an evaluator assessing AI Agent responses.\n\nThe Agent represents Parag Agrawal on their website. Evaluate if the response is acceptable based on:\n1. **Accuracy** - Does it align with Parag Agrawal's actual background?\n2. **Professionalism** - Is the tone appropriate?\n3. **Completeness** - Does it answer the question?\n\n## Parag Agrawal's Background:\nParag Agrawal leads a balanced life that blends his demanding career with a strong emphasis on family and personal interests. He is married to Vineeta Agarwala, a general partner at the venture capital firm Andreessen Horowitz, where she invests in biotech and health tech. Together, they reside in San Francisco and are raising two children. Despite his high-profile career, Parag values quality time with his family and enjoys nurturing curiosity both personally and professionally. Outside of work, he has interests in reading, mentoring young talent, and staying connected with his cultural roots. His personal life reflects a g

In [35]:
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [ ]:
def evaluator_user_prompt(message, reply, history): 
    user_prompt = (
        "You are evaluating the most recent response from an AI Agent in the context of a conversation.\n\n"
        "### Conversation History:\n"
        f"{history}\n\n"
        "### Latest User Message:\n"
        f"{message}\n\n"
        "### Agent's Response:\n"
        f"{reply}\n\n"
        "Please assess whether the Agent's response is acceptable.\n"
    )
    return user_prompt

In [37]:
from openai import OpenAI
# openai_python_client = OpenAI(api_key=openai_api_key, base_url=deepseek_base_URL)
openai_python_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=openai_api_key
)

print("7. OpenAI client created:", openai_python_client)

7. OpenAI client created: <openai.OpenAI object at 0x000002744AD9BB10>


In [38]:
def evaluate(message, reply, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(message, reply, history)}]
    response = openai_python_client.chat.completions.parse(model="google/gemma-3-27b-it", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [39]:
# Submit a User Prompt to LLM1 - DeepSeek
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a Degree?"}]
response = openai_python_client.chat.completions.create(
        model="deepseek/deepseek-chat",
        messages=messages,
        stream=False
    )

reply = response.choices[0].message.content

In [40]:
reply

"Yes, I hold a B.Tech in Computer Science and Engineering from the Indian Institute of Technology (IIT) Bombay, one of India's premier engineering institutions. After that, I pursued a Ph.D. in Computer Science at Stanford University, where my research focused on uncertainty in data management and integration. My academic background has been foundational in shaping my technical expertise and career trajectory.  \n\nWould you like to know more about my research or how my education influenced my career?"

In [42]:
# Evaluate LLM1- Deepseek Response via LLM2 - Gemini
evaluate("do you hold a Degree?", reply, messages[:1])

In [19]:
def reRun(message, reply, history, feedback):
    updated_system_prompt= (
    system_prompt
    + "\n\n## Previous Answer Rejected\n"
      "Your most recent reply was rejected by the quality control system.\n"
    + f"\n### Your Attempted Answer:\n{reply}\n"
    + f"\n### Reason for Rejection:\n{feedback}\n"
    + "\nPlease revise your response to meet quality expectations, maintaining a professional, helpful, and engaging tone."
    )
    
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai_python_client.chat.completions.create(
        model="deepseek/deepseek-chat",
        messages=messages,
        stream=False
    )

    return response.choices[0].message.content


In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai_python_client.chat.completions.create(
        model="deepseek/deepseek-chat",
        messages=messages,
        stream=False
    )
    reply = response.choices[0].message.content

    evaluation = evaluate(message, reply, history)
    # evaluation = evaluate(message, reply, [])
    print(evaluation.is_acceptable)

    if evaluation.is_acceptable:
        print("Passed evaluation of LLM1 - returning reply")
    else:
        print("Failed evaluation of LLM1 - retrying")
        print(evaluation.feedback)
        reply = reRun(message, reply, history, evaluation.feedback)
        
    return reply

In [ ]:
gr.ChatInterface(chat,
    title="Chat with Parag Agrawal",
    description="Ask me anything about my career, background, skills, and experience!",
).launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


False
Failed evaluation of LLM1 - retrying
The Agent’s response is highly unprofessional and lacks basic politeness. Using “u” instead of “you” is too informal for an agent representing a professional like Parag Agrawal. The response is also abrupt and doesn’t acknowledge the detailed information provided by the user, nor does it maintain an engaging and thoughtful tone as expected. It feels more like a demand for information than a continuation of a conversation.
False
Failed evaluation of LLM1 - retrying
The agent simply repeats the user's statement (
